In [39]:
res = !gcloud config get core/project
PROJECT_ID = res[0]
SERVICE_ACCOUNT = 'gemma-vertexai-chatbot@able-analyst-416817.iam.gserviceaccount.com'
from datetime import datetime
CONTAINER_IMAGE_NAME="gemma-chatbot"
GCP_REGION='us-central1'
TAG_NAME = 'masterv9'
KAGGLE_USERNAME='andrehpereh1'
KAGGLE_KEY='5859e39806d9456749dcbac685f04bc9'
CONTAINER_IMAGE_NAME_DATA_PREP = f"{CONTAINER_IMAGE_NAME}-data-preparation"
CONTAINER_IMAGE_NAME_FINE_TUNE = f"{CONTAINER_IMAGE_NAME}-fine-tunning"
CONTAINER_IMAGE_NAME_RUN_APP = f"{CONTAINER_IMAGE_NAME}-running-app"
CONTAINER_IMAGE_NAME_PIPELINE = f"{CONTAINER_IMAGE_NAME}-pipeline-app"
CONTAINER_IMAGE_RUNNING_APP = f"{CONTAINER_IMAGE_NAME}-running-app"
BUCKET_NAME = 'able-analyst-416817-chatbot-v1'
# BUCKET_URI = f"gs://{BUCKET_NAME}"
FINE_TUNE_FLAG=  False
EPOCHS=12
MODEL_NAME="gemma_2b_en"

substitutions="""
_CONTAINER_IMAGE_NAME_PIPELINE={},\
TAG_NAME={},\
_BUCKET_NAME={},\
_FINE_TUNE_FLAG={},\
_EPOCHS={},\
_MODEL_NAME={}
""".format(
           CONTAINER_IMAGE_NAME_PIPELINE,
           TAG_NAME,
           BUCKET_NAME,
           FINE_TUNE_FLAG,
           EPOCHS,
           MODEL_NAME
           ).strip()
print(substitutions)
!gcloud builds submit . --config "components/pipeline/cloudbuild.yaml" --substitutions {substitutions} --region={GCP_REGION}


_CONTAINER_IMAGE_NAME_PIPELINE=gemma-chatbot-pipeline-app,TAG_NAME=masterv9,_BUCKET_NAME=able-analyst-416817-chatbot-v1,_FINE_TUNE_FLAG=False,_EPOCHS=12,_MODEL_NAME=gemma_2b_en
Creating temporary tarball archive of 69 file(s) totalling 830.5 KiB before compression.
Uploading tarball of [.] to [gs://able-analyst-416817_cloudbuild/source/1712146109.745814-f08bb80490184a9ca23e3c03c8121196.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/able-analyst-416817/locations/us-central1/builds/6d3ab1e4-38c2-4335-a43e-54fea62d80c6].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds;region=us-central1/6d3ab1e4-38c2-4335-a43e-54fea62d80c6?project=24796876098 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "6d3ab1e4-38c2-4335-a43e-54fea62d80c6"

FETCHSOURCE
Fetching storage object: gs://able-analyst-416817_cloudbuild/source/1712146109.745814-f08bb80490184a9ca23e3c03c8121196.tgz#1712146110415685
Copying gs://able-an

In [ ]:
import os

CONTAINER_IMAGE_NAME="gemma-chatbot"
GCP_REGION = 'us-central1'
TAG_NAME = 'masterv4'

CONTAINER_IMAGE_NAME_PIPELINE = f"{CONTAINER_IMAGE_NAME}-pipeline-app"
CONTAINER_IMAGE_RUNNING_APP = f"{CONTAINER_IMAGE_NAME}-running-app"

#,_CONTAINER_IMAGE_RUNNING_APP={CONTAINER_IMAGE_RUNNING_APP}
substitutions=f"_CONTAINER_IMAGE_NAME_PIPELINE={CONTAINER_IMAGE_NAME_PIPELINE},TAG_NAME={TAG_NAME}"

In [ ]:
!gcloud builds submit . --config "./cloudbuil_cloudrun.yaml" --substitutions {substitutions} --region=us-central1

In [ ]:

!gcloud functions deploy hello_world_trigger \
--gen2 \
--region='us-central1' \
--runtime=python39 \
--source='./components/cloud_functions/' \
--entry-point='trigger_pipeline_cloud_function' \
--trigger-topic='your-pipeline-trigger-topic' \
--memory='2048MB'

In [ ]:
import kfp
kfp.__version__
dir(kfp.dsl)

In [ ]:
from components.cloud_functions.main import trigger_pipeline_cloud_function
import json

data = {
    "user_name": "andrehpereh",
    "files": [
        {"file_path": "gs://personalize-chatbots-v1/andrehpereh/input_data/WhatsApp Chat with Anki.txt", "filename": "WhatsApp Chat with Anki.txt"},
        {"file_path": "gs://personalize-chatbots-v1/andrehpereh/input_data/WhatsApp Chat with Ilse Flatmate.txt", "filename": "WhatsApp Chat with Ilse Flatmate.txt"},
        {"file_path": "gs://personalize-chatbots-v1/andrehpereh/input_data/WhatsApp Chat with Michael.txt", "filename": "WhatsApp Chat with Michael.txt"}
    ],
    "blob_folder": "andrehpereh/input_data",
    "model_name": "gemma_2b_en",
    "epochs": "12",
    "bucket_name": "personalize-chatbots-v1",
    "project_id": "able-analyst-416817"
}

# Convert dictionary to JSON string
json_data = json.dumps(data)

# Encode JSON string to bytes
data_bytes = json_data.encode('utf-8')

# Pass the encoded bytes to the function
trigger_pipeline_cloud_function()


In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "andrehpereh1"
os.environ['KAGGLE_KEY'] = "5859e39806d9456749dcbac685f04bc9"


In [ ]:
import importlib
import util
importlib.reload(util)
model_paths_and_config = util.get_model_paths_and_config(Config.MODEL_NAME)
model_paths_and_config

In [ ]:
Config.TRAIN_DATA_DIR, Config.BUCKET_NAME

In [ ]:
data = process_whatsapp_chat(Config.BUCKET_NAME, Config.TRAIN_DATA_DIR)

In [ ]:
model_paths_and_config

In [ ]:
finetuned_weights_path = finetune_gemma(data=data[:50], model_paths=model_paths_and_config, fine_tune_flag=False, model_name=Config.MODEL_NAME, rank_lora=Config.SEQUENCE_LENGTH, sequence_length=Config.SEQUENCE_LENGTH, epochs=Config.EPOCHS, batch_size=Config.BATCH_SIZE)

In [ ]:
from numba import cuda
device = cuda.get_current_device()
cuda.select_device(device.id)
cuda.close()

In [ ]:
type(finetuned_weights_path)
print(model_paths_and_config['finetuned_weights_path'])
print(model_paths_and_config['finetuned_model_dir'])

In [ ]:
!gsutil cp gs://able-analyst-416817-chatbot-v1/gemma_2b_en_raw/gemma_2b_en/model.weights.h5 ./test

In [ ]:
finetuned_weights_path.save_weights("gs://able-analyst-416817-chatbot-v1/gemma_2b_en_raw/gemma_2b_en/test/model.weights.h5")
finetuned_weights_path.preprocessor.tokenizer.save_assets("gs://able-analyst-416817-chatbot-v1/gemma_2b_en_raw/gemma_2b_en/test")

In [ ]:
output_dir = convert_checkpoints(
    weights_file=model_paths_and_config['finetuned_weights_path'],
    size=model_paths_and_config['model_size'],
    output_dir=model_paths_and_config['huggingface_model_dir'],
    vocab_path=model_paths_and_config['finetuned_vocab_path'],
)


In [ ]:
output_dir=model_paths_and_config['huggingface_model_dir']

In [ ]:
model_paths_and_config, Config.BUCKET_NAME
bucket_name, blob_name = os.path.dirname("gs://able-analyst-416817-chatbot-v1/gemma_2b_en_raw/gemma_2b_en/model.weights.h5").lstrip("gs://").split("/", 1) 
bucket_name, blob_name

In [ ]:
os.path.basename("gs://able-analyst-416817-chatbot-v1/gemma_2b_en_raw/gemma_2b_en/model.weights.h5")

In [ ]:
destination_path = upload2bs(local_directory = output_dir, bucket_name = Config.BUCKET_NAME, destination_subfolder = model_paths_and_config['deployed_model_blob'])

In [ ]:
import importlib
import util
importlib.reload(util)

util.download_all_from_blob(Config.BUCKET_NAME, "gemma_2b_en_raw/gemma_2b_en", './')

In [ ]:
Config.BUCKET_NAME
model_paths_and_config['huggingface_model_dir']

In [ ]:
model_paths_and_config['deployed_model_blob']

In [ ]:
aiplatform.init(project=Config.PROJECT_ID, location=Config.REGION, staging_bucket=Config.BUCKET_URI)

In [ ]:
import datetime
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20240220_0936_RC01"


def get_job_name_with_datetime(prefix: str) -> str:
    suffix = datetime.datetime.now().strftime("_%Y%m%d_%H%M%S")
    return f"{prefix}{suffix}"


def deploy_model_vllm(
    model_name: str,
    model_uri: str,
    service_account: str,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    max_model_len: int = 8192,
    dtype: str = "bfloat16",
) -> tuple[aiplatform.Model, aiplatform.Endpoint]:
    # Upload the model to "Model Registry"
    job_name = get_job_name_with_datetime(model_name)
    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        "--gpu-memory-utilization=0.95",
        f"--max-model-len={max_model_len}",
        f"--dtype={dtype}",
        "--disable-log-stats",
    ]
    model = aiplatform.Model.upload(
        display_name=job_name,
        artifact_uri=model_uri,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_command=["python", "-m", "vllm.entrypoints.api_server"],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
    )

    # Deploy the model to an endpoint to serve "Online predictions"
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )

    return model, endpoint

In [ ]:
# Set up artificially since the model was already in a bucket
model_paths_and_config['deployed_model_uri'] = 'gs://able-analyst-416817-chatbot-v1/gemma_2b_en/20240314162107'
model_paths_and_config['model_name_vllm']

In [ ]:
max_model_len = 2048

model, endpoint = deploy_model_vllm(
    model_name=model_paths_and_config['model_name_vllm'],
    model_uri=model_paths_and_config['deployed_model_uri'],
    service_account=Config.SERVICE_ACCOUNT,
    machine_type=model_paths_and_config['machine_type'],
    accelerator_type=model_paths_and_config['accelerator_type'],
    accelerator_count=model_paths_and_config['accelerator_count'],
    max_model_len=max_model_len,
)

In [ ]:
TEST_EXAMPLES = [
    "What is the plan for tonight?",
    "What would you like to drink?",
    "Are you coming tonight?"
]

# Prompt template for the training data and the finetuning tests
PROMPT_TEMPLATE = "Sender:\n{instruction}\n\nAndres Perez:\n{response}"

TEST_PROMPTS = [
    PROMPT_TEMPLATE.format(instruction=example, response="")
    for example in TEST_EXAMPLES
]

def test_vertexai_endpoint(endpoint: aiplatform.Endpoint):
    for question, prompt in zip(TEST_EXAMPLES, TEST_PROMPTS):
        instance = {
            "prompt": prompt,
            "max_tokens": 56,
            "temperature": 0.0,
            "top_p": 1.0,
            "top_k": 1,
            "raw_response": True,
        }
        response = endpoint.predict(instances=[instance])
        output = response.predictions[0]
        print(f"{question}\n{output}\n{'- '*40}")


test_vertexai_endpoint(endpoint)

In [ ]:
res = !gcloud config get core/project
PROJECT_ID = res[0]
SERVICE_ACCOUNT = 'gemma-vertexai-chatbot@able-analyst-416817.iam.gserviceaccount.com'
from datetime import datetime
CONTAINER_IMAGE_NAME="gemma-chatbot"
GCP_REGION='us-central1'
IMAGE_NAME="gemma-chatbot"
TAG_NAME = 'latest'
KAGGLE_USERNAME='andrehpereh1'
KAGGLE_KEY='5859e39806d9456749dcbac685f04bc9'

In [ ]:
SUBSTITUTIONS="""
_CONTAINER_IMAGE_NAME={},\
TAG_NAME={}\
""".format(
           f"{CONTAINER_IMAGE_NAME}-data-preparation",
           TAG_NAME, 
           ).strip()
print(SUBSTITUTIONS)


In [ ]:
# Runs the data_preparation component image. (Development, when tested should be moved to the main cloudbuild in the project folder)
# Pay attention to the "." after summit. Might need some changes when move to the master pipeline.
!gcloud builds submit . --timeout=15m --config "components/data_preparation/cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}
# DO not forget the tag
#!docker run gcr.io/able-analyst-416817/gemma-chatbot-data-preparation:latest data_ingestion.py --bucket-name 'able-analyst-416817-chatbot-v1' --directory 'input_data/andrehpereh'

In [ ]:
SUBSTITUTIONS="""
_CONTAINER_IMAGE_NAME={},\
_KAGGLE_USERNAME={},\
_KAGGLE_KEY={},\
TAG_NAME={}\
""".format(
           f"{CONTAINER_IMAGE_NAME}-fine-tunning",
           KAGGLE_USERNAME,
           KAGGLE_KEY,
           TAG_NAME, 
           ).strip()
print(SUBSTITUTIONS)

# Builds image
!gcloud builds submit . --config "components/fine_tunning/cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}


In [ ]:
import json

data = ["Sender: FoooodddAndres Perez: Coming :)", "Sender: Can I maybe borrow your iron? Andres Perez: It\'s not my iron But yeah haha Or is it?"]
model_paths = """{"finetuned_model_dir": "./gemma_2b_en", "finetuned_weights_path": "./gemma_2b_en/model.weights.h5"}"""
print(len(data))
model_paths = json.dumps(model_paths)
#!docker run gcr.io/able-analyst-416817/gemma-chatbot-fine-tunning:latest trainer.py --data {data} --model-paths {model_paths}

In [ ]:
!python ./components/fine_tunning/trainer.py --data {data} --model-paths {model_paths}

In [ ]:
model_paths_and_config['huggingface_model_dir']

In [ ]:
!python ./components/fine_tunning/conversion_function.py --weights-file {model_paths_and_config['finetuned_weights_path']} --size {model_paths_and_config['model_size']} --vocab-path {model_paths_and_config['finetuned_vocab_path']} --output-dir {model_paths_and_config['huggingface_model_dir']}

In [ ]:
output_dir = convert_checkpoints(
    weights_file=model_paths_and_config['finetuned_weights_path'],
    size=model_paths_and_config['model_size'],
    output_dir=model_paths_and_config['huggingface_model_dir'],
    vocab_path=model_paths_and_config['finetuned_vocab_path'],
)

In [ ]:
import json
SUBSTITUTIONS="""
_CONTAINER_IMAGE_NAME={},\
TAG_NAME={}\
""".format(
           f"{CONTAINER_IMAGE_NAME}-experimental",
           TAG_NAME,
           ).strip()
print(SUBSTITUTIONS)
data = '"[\\"Sender: FoooodddAndres Perez: Coming :)\\", \\"Sender: Can I maybe borrow your iron? Andres Perez: It\'s not my iron But yeah haha Or is it?\\"]"'
model_paths = """{"finetuned_model_dir": "./gemma_2b_en", "finetuned_weights_path": "./gemma_2b_en/model.weights.h5"}"""
data_json = json.dumps(data)
model_paths_json = json.dumps(model_paths)
#!gcloud builds submit . --timeout=15m --config "components/experimental/cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}
!docker run gcr.io/able-analyst-416817/gemma-chatbot-experimental:latest experimental.py {data_json} {model_paths_json}


In [ ]:
#Re-runs the image to restart the website, service account might be needed with this one.  (Development, when tested should be moved to the main cloudbuild in the project folder)
SUBSTITUTIONS="""
_CONTAINER_IMAGE_NAME={},\
_GCP_REGION={},\
TAG_NAME={}\
""".format(
           f"{CONTAINER_IMAGE_NAME}-running-app",
           GCP_REGION,
           TAG_NAME, 
           ).strip()
print(SUBSTITUTIONS)
!gcloud builds submit . --timeout=15m --config "components/app_flask/cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}
#!gcloud builds submit . --timeout=15m --config "cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}


In [ ]:
SUBSTITUTIONS="""
_CONTAINER_IMAGE_NAME={},\
_GCP_REGION={},\
TAG_NAME={}\
""".format(
           f"{CONTAINER_IMAGE_NAME}-running-app",
           GCP_REGION,
           TAG_NAME, 
           ).strip()
print(SUBSTITUTIONS)
!gcloud builds submit . --timeout=15m --config "components/app_flask/cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}
#!gcloud builds submit . --timeout=15m --config "cloudbuild.yaml" --substitutions {SUBSTITUTIONS} --region={GCP_REGION}

In [ ]:
from config import Config
from util import get_model_paths_and_config, upload2bs

res = !gcloud config get core/project
PROJECT_ID = res[0]
from google.cloud import compute_v1

project = PROJECT_ID  # Replace with your project ID

# Regions to consider
regions = ["us-central1", "europe-west4", "asia-east1"]  

client = compute_v1.AcceleratorTypesClient()

for region in regions:
    zone_client = compute_v1.ZonesClient()
    all_zones = zone_client.list(project=project)

    zone_list = [zone for zone in all_zones if zone.region == f"regions/{region}"]

    for zone in zone_list:
        result = client.describe(
            project=project, zone=zone.name, accelerator_type="nvidia-l4"
        )
        # If the result isn't an error, the 'nvidia-l4' type is available
        GCP_REGION = region  # Update your region variable
        break 

In [ ]:
!bq mk chatbot

In [ ]:
from google.cloud import bigquery
import os

# Construct a BigQuery client object
os.environ['PROJECT_ID'] = 'able-analyst-416817'
client = bigquery.Client(os.environ.get('PROJECT_ID'))

# Define your dataset and table information
project_id = os.environ.get('PROJECT_ID')
project = os.environ.get('PROJECT_ID')

In [ ]:

dataset_id = "chatbot"
table_id = "users" 

# Schema definition
schema = [
    # bigquery.SchemaField("user_id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("email", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("password_hash", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("created_at", "TIMESTAMP", description="Record creation timestamp")  
    #bigquery.SchemaField("salt", "STRING", mode="REQUIRED"),
    #bigquery.SchemaField("created_at", "TIMESTAMP", mode="REQUIRED"),
    #bigquery.SchemaField("last_login", "TIMESTAMP", mode="NULLABLE") 
]

# Create a table object
table = bigquery.Table(project_id + "." + dataset_id + "." + table_id, schema=schema)

# Create the table in BigQuery
table = client.create_table(table)  
print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))


In [ ]:
dataset_id = "chatbot"
table_id = "user_training_status" 

In [ ]:
# Schema definition
schema = [
    bigquery.SchemaField("email", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("training_status", "BOOL", mode="REQUIRED"),
    bigquery.SchemaField("end_point", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("created_at", "TIMESTAMP", description="Record creation timestamp")  
]

# Create a table object
table = bigquery.Table(project_id + "." + dataset_id + "." + table_id, schema=schema)

# Create the table in BigQuery
table = client.create_table(table)  
print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))

In [ ]:
dataset_id = "chatbot"
table_id = "users_endpoints" 

In [ ]:
schema = [
    bigquery.SchemaField("email", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("training_status", "BOOL", mode="REQUIRED"),
    bigquery.SchemaField("end_point", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("created_at", "TIMESTAMP", description="Record creation timestamp")  
]

# Create a table object
table = bigquery.Table(project_id + "." + dataset_id + "." + table_id, schema=schema)

# Create the table in BigQuery
table = client.create_table(table)  
print("Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id))

In [ ]:
from kfp import dsl

In [ ]:
from google.cloud import bigquery
#This part can be wrapped in a function.
query = f"""
UPDATE `{os.environ.get('PROJECT_ID')}.{DATASET_ID}.{USER_TRAINING_STATUS}`
SET end_point = '{endpoint_resource}'  
WHERE email = '{email}' 
"""
# Create a query job configuration
job_config = bigquery.QueryJobConfig()

# Execute the update query
query_job = client.query(query, job_config=job_config)
query_job.result() 

print("End point has been stored.")


In [ ]:
project

In [ ]:
import os
from google.cloud import bigquery
DATASET_ID = 'chatbot' # This should be moved to a config file
USER_TRAINING_STATUS = 'user_training_status' # This should be moved to a config file
#This part can be wrapped in a function
email = 'andreshperesh@gmail.com'

resource_uri = "https://us-central1-aiplatform.googleapis.com/v1/projects/24796876098/locations/us-central1/endpoints/2572265671740096512/operations/6634221199406661632"

print("This is the passed end pooint", resource_uri)

print("This is the project", project)

client = bigquery.Client(project)
print("This is the client", client)
table_ref = client.dataset(DATASET_ID).table(USER_TRAINING_STATUS)
table = client.get_table(table_ref)
row_to_insert = {
    'email': email,
    'end_point': resource_uri,
    'training_status': True
}
client.insert_rows(table, [row_to_insert]) 
errors = client.insert_rows(table, [row_to_insert])
if errors:  # Check if there were errors
    print("The model has been trained, but error updating resource_uri for {}: {}".format(email, errors))
else:
    print("User training has been updated")
print("End point has been stored.")


In [ ]:
import logging
import re  

# ... rest of your imports ...

def predict_custom_trained_model_sample(
  project: str,
  endpoint_id: str,
  location: str = "us-central1",
  api_endpoint: str = "us-central1-aiplatform.googleapis.com",
  user_input: str = input
):
    
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    logging.basicConfig(level=logging.INFO)  # Set up basic logging

    prompt_input = f"Sender:\n{user_input}\n\nAndres Perez:\n",
    instances = {'prompt': prompt_input[0], 'max_tokens': 256, 'temperature': 1.4, 'top_p': 0.8, 'top_k': 4}

    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)

    try: 
        # The format of each instance should conform to the deployed model's prediction input schema.
        instances = instances if isinstance(instances, list) else [instances]
        instances = [
          json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
        ]
        parameters_dict = {}
        parameters = json_format.ParseDict(parameters_dict, Value())
        endpoint = client.endpoint_path(
          project=project, location=location, endpoint=endpoint_id
        )

        response = client.predict(
          endpoint=endpoint, instances=instances, parameters=parameters
        )

        # The predictions are a google.protobuf.Value representation of the model's predictions.
        predictions = response.predictions
        pattern = r"Perez:\nOutput:\n(.*)"
        match = re.search(pattern, predictions[0])

    if match:
        return match.group(1)
    else:
        logging.warning("Prediction not found in the response")
        return "Error: Prediction not found in the response."

    except Exception as e:  # Log any exceptions
        logging.error("Prediction failed: %s", e) 
        return "Error: Problem generating response."


In [ ]:
import re

string = "https://us-central1-aiplatform.googleapis.com/v1/projects/24796876098/locations/us-central1/endpoints/2572265671740096512/operations/6634221199406661632"

pattern = r"\/projects\/([^\/]+)\/locations\/([^\/]+)\/endpoints\/([^\/]+)\/operations\/([^\/]+)"

match = re.search(pattern, string)

if match:
    project_id = match.group(1)
    location = match.group(2)
    endpoint_id = match.group(3)
    operation_id = match.group(4)

    print("Project ID:", project_id)
    print("Location:", location)
    print("Endpoint ID:", endpoint_id)

In [ ]:
with open('Wednesday at 12-33 AM.txt', 'r') as file:
    contents = file.read()
    print(contents)

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, ChatSession

project_id = project_id
location = "us-central1"
vertexai.init(project=project_id, location=location)
model = GenerativeModel("gemini-1.0-pro")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

In [ ]:

from vertexai.preview.generative_models import GenerativeModel
gemini_pro_model = GenerativeModel("gemini-1.0-pro")

In [ ]:
from vertexai import generative_models 
import vertexai
vertexai.__version__

In [7]:
from google.cloud import bigquery
import os

# Construct a BigQuery client object
os.environ['PROJECT_ID'] = 'able-analyst-416817'
client = bigquery.Client(os.environ.get('PROJECT_ID'))

# Define your dataset and table information
project_id = os.environ.get('PROJECT_ID')
project = os.environ.get('PROJECT_ID')

In [ ]:
from components.data_preparation import data_ingestion
yup = data_ingestion.data_preparation(
    bucket_name = 'personalize-chatbots-v1', directory = 'andrehpereh/input_data/',
    gemini_pro_model= gemini_pro_model, pair_count=6, data_augmentation_iter=1
)

In [28]:

def translate_text(target: str, text: str) -> dict:
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    if isinstance(text, bytes):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)

    print("Text: {}".format(result["input"]))
    print("Translation: {}".format(result["translatedText"]))
    print("Detected source language: {}".format(result["detectedSourceLanguage"]))

    return result



In [29]:
with open('./input_data/whatsapp_spanish.txt', 'r') as f:
    lines = f.readlines()

lines[3]

'24/05/23 10:28\u202fa.\u202fm. - +52 246 311 9472: El Jueves de la Próxima semana Sale mi Vuelo, Fui a una conferencia Que hubo en puebla y en ella nos dijeron que Teníamos que notificar a nuestros reclutadores 1 hora antes de irnos al Camp\n'

Text: 24/05/23 10:28 a. m. - +52 246 311 9472: El Jueves de la Próxima semana Sale mi Vuelo, Fui a una conferencia Que hubo en puebla y en ella nos dijeron que Teníamos que notificar a nuestros reclutadores 1 hora antes de irnos al Camp

Translation: 05/24/23 10:28 a.m. m. - +52 246 311 9472: Next week my flight leaves, I went to a conference in Puebla and they told us that we had to notify our recruiters 1 hour before leaving for Camp.

Detected source language: es


{'translatedText': '05/24/23 10:28 a.m. m. - +52 246 311 9472: Next week my flight leaves, I went to a conference in Puebla and they told us that we had to notify our recruiters 1 hour before leaving for Camp.\n',
 'detectedSourceLanguage': 'es',
 'input': '24/05/23 10:28\u202fa.\u202fm. - +52 246 311 9472: El Jueves de la Próxima semana Sale mi Vuelo, Fui a una conferencia Que hubo en puebla y en ella nos dijeron que Teníamos que notificar a nuestros reclutadores 1 hora antes de irnos al Camp\n'}

In [ ]:
# Imports the Google Cloud Translation library
from google.cloud import translate


# Initialize Translation client
def translate_text(
    text: str = "YOUR_TEXT_TO_TRANSLATE", project_id: str = "YOUR_PROJECT_ID"
) -> translate.TranslationServiceClient:
    """Translating Text."""

    client = translate.TranslationServiceClient()

    location = "global"

    parent = f"projects/{project_id}/locations/{location}"

    # Translate text from English to French
    # Detail on supported types can be found here:
    # https://cloud.google.com/translate/docs/supported-formats
    response = client.translate_text(
        request={
            "parent": parent,
            "contents": [text],
            "mime_type": "text/plain",  # mime types: text/plain, text/html
            "source_language_code": "en-US",
            "target_language_code": "fr",
        }
    )

    # Display the translation for each input text provided
    for translation in response.translations:
        print(f"Translated text: {translation.translated_text}")

    return response


In [ ]:
def list_languages_with_target(target: str) -> dict:
    """Lists all available languages and localizes them to the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    results = translate_client.get_languages(target_language=target)

    for language in results:
        print("{name} ({language})".format(**language))

    return results
list_languages_with_target('en')

In [2]:
def list_languages() -> dict:
    """Lists all available languages."""
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()

    results = translate_client.get_languages()

    for language in results:
        print("{name} ({language})".format(**language))

    return results




In [23]:
from google.cloud import storage
bucket_name = 'personalize-chatbots-v1'
blob_name = 'hermi_test/hermi_whatsapp.txt'
storage_client = storage.Client()
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(blob_name)
contents = blob.download_as_string().decode('utf-8')


In [24]:
len(contents)

11847

In [25]:
contents[0]

'0'

In [ ]:
res = translate_text('en', contents)


In [37]:
print(res["translatedText"][350:750])

3 10:27 a.m. m. - +52 246 311 9472: Hello, good day 05/24/23 10:28 a. m. - +52 246 311 9472: My flight leaves on Thursday next week, I went to a conference in Puebla and they told us that we had to notify our recruiters 1 hour before leaving for Camp 05/24/23 11 :40 a.m. m. - Iena mx: Hello 05/24/23 11:40 a. m. - Iena mx:<Multimedia omitido> 05/24/23 11:40 a.m. m. - Iena mx: This is the only thing
